## Using the module "excel_report_creator" for solving practical tasks

To demonstrate the work of the module "excel_report_creator" we will build sales review report of a British company.  
This dataset is free and has been downloaded from Kaggle.


Suppose, this company has several sales departments and each of them controls a specified region.  
Head of this department and his employees need to receive analytic reports to correct their work, to change effort point and to manage the sales.
The company has adopted certain reporting standards in Excel.  
The company has an analyst. He should create reports in Excel format on demand of the sales department. There are several typical forms of reports in the company and the report we are building now should be one of these forms.  
See target report form in the file: *sales_report.xlsx*  

To build this report the analyst has two ways:
1) to prepare all tables and save them into Excel, then merge them into 1 file and edit according to the format required;
2) to create a template for this report once and then create reports of this form automatically using the module "excel_report_creator" by several clicks.

Obviously, the second way is preferable for regular work, because it saves time and excludes human mistakes. Next we will implement this variant of task solution.



In [1]:
import pandas as pd
import excel_report_creator
from excel_report_creator import ReportCreator
import datetime


In [2]:
# func for defining a previous month
def month_shift(month_name: str, days_offset: int):
    d = datetime.datetime.strptime(month_name, '%B')
    d += datetime.timedelta(days=days_offset)
    return d.strftime('%B')

Loading dataset with example data.

In [3]:
df = pd.read_csv('british_sells_data.csv', sep=',', encoding='cp1251')

In [4]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


To build the target report we need to filter the source dataset by 3 parameters: year, month, country.  
Now we are defining them by random.

In [6]:
params = {'month': 'June', 'year': 2011, 'country': 'France'}

We also need to difine a name of the month before the current month to calculate the dynamics.

In [7]:
params['previous_month'] = month_shift(params['month'], -28)
params

{'month': 'June', 'year': 2011, 'country': 'France', 'previous_month': 'May'}

Adding needed columns to dataset.

In [8]:
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64[s]')
df['Month'] = df['InvoiceDate'].dt.month_name()
df['Year'] = df['InvoiceDate'].dt.year
df['Day'] = df['InvoiceDate'].dt.day
df['Amount'] = df['Quantity'] * df['UnitPrice']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Year,Day,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,2010,1,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,2010,1,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,2010,1,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,2010,1,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,2010,1,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,December,2011,9,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,December,2011,9,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,December,2011,9,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,December,2011,9,16.60


Filtering dataset to obtain slices of the current and previous months.

In [9]:
current_month = df[(df['Month'] == params['month'])&(df['Year'] == params['year'])&(df['Country']==params['country'])]
current_month

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Year,Day,Amount
213273,555531,POST,POSTAGE,11,2011-06-05 11:53:00,18.00,12678.0,France,June,2011,5,198.00
213274,555531,22728,ALARM CLOCK BAKELIKE PINK,24,2011-06-05 11:53:00,3.75,12678.0,France,June,2011,5,90.00
213275,555531,22846,BREAD BIN DINER STYLE RED,4,2011-06-05 11:53:00,14.95,12678.0,France,June,2011,5,59.80
213276,555531,23280,FOLDING BUTTERFLY MIRROR HOT PINK,96,2011-06-05 11:53:00,0.72,12678.0,France,June,2011,5,69.12
213277,555531,23203,JUMBO BAG DOILEY PATTERNS,200,2011-06-05 11:53:00,1.79,12678.0,France,June,2011,5,358.00
...,...,...,...,...,...,...,...,...,...,...,...,...
242328,558334,21086,SET/6 RED SPOTTY PAPER CUPS,24,2011-06-28 13:11:00,0.65,12599.0,France,June,2011,28,15.60
242329,558334,21094,SET/6 RED SPOTTY PAPER PLATES,12,2011-06-28 13:11:00,0.85,12599.0,France,June,2011,28,10.20
242330,558334,22544,MINI JIGSAW SPACEBOY,24,2011-06-28 13:11:00,0.42,12599.0,France,June,2011,28,10.08
242331,558334,POST,POSTAGE,1,2011-06-28 13:11:00,18.00,12599.0,France,June,2011,28,18.00


In [10]:
previous_month =  df[(df['Month'] == params['previous_month'])&(df['Year'] == params['year'])&(df['Country']==params['country'])]
previous_month

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Year,Day,Amount
172461,551539,20719,WOODLAND CHARLOTTE BAG,10,2011-05-03 08:40:00,0.85,12598.0,France,May,2011,3,8.50
172462,551539,20723,STRAWBERRY CHARLOTTE BAG,10,2011-05-03 08:40:00,0.85,12598.0,France,May,2011,3,8.50
172463,551539,23204,CHARLOTTE BAG APPLES DESIGN,20,2011-05-03 08:40:00,0.85,12598.0,France,May,2011,3,17.00
172464,551539,20724,RED RETROSPOT CHARLOTTE BAG,20,2011-05-03 08:40:00,0.85,12598.0,France,May,2011,3,17.00
172465,551539,22661,CHARLOTTE BAG DOLLY GIRL DESIGN,10,2011-05-03 08:40:00,0.85,12598.0,France,May,2011,3,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...
208163,555096,22989,SET 2 PANTRY DESIGN TEA TOWELS,6,2011-05-31 13:32:00,3.25,12682.0,France,May,2011,31,19.50
208164,555096,21700,BIG DOUGHNUT FRIDGE MAGNETS,12,2011-05-31 13:32:00,0.85,12682.0,France,May,2011,31,10.20
208165,555096,23155,KNICKERBOCKERGLORY MAGNET ASSORTED,12,2011-05-31 13:32:00,0.83,12682.0,France,May,2011,31,9.96
208166,555096,POST,POSTAGE,3,2011-05-31 13:32:00,18.00,12682.0,France,May,2011,31,54.00


Processing data and obtaining needed tables

In [11]:
t1 = pd.DataFrame(current_month.agg({'Amount': 'sum', 'Quantity': 'sum', 'Description': 'nunique', 'CustomerID': 'nunique', 'InvoiceNo': 'nunique'}), columns=['current_results'])
t1['previous_results'] = previous_month.agg({'Amount': 'sum', 'Quantity': 'sum', 'Description': 'nunique', 'CustomerID': 'nunique', 'InvoiceNo': 'nunique'})
t1.loc['Average_bill'] = t1.loc['Amount'] / t1.loc['InvoiceNo']
t1.loc['Average_price'] = t1.loc['Amount'] / t1.loc['Quantity']
t1['sales_growth'] = t1['current_results']/ t1['previous_results'] - 1
t1

,current_results,previous_results,sales_growth
Amount,15991.870000,17527.080000,-0.087591
Quantity,9441.000000,9780.000000,-0.034663
Description,333.000000,382.000000,-0.128272
CustomerID,25.000000,28.000000,-0.107143
InvoiceNo,34.000000,43.000000,-0.209302
Average_bill,470.349118,407.606512,0.153929
Average_price,1.693875,1.792135,-0.054829


In [12]:
days_list = [x for x in range(1, 32)]
t2 = pd.DataFrame(index=days_list)
t2 = t2.join(current_month.groupby('Day').agg({'Amount': 'sum', 'CustomerID': 'nunique'}))
t2 = t2.join(previous_month.groupby('Day').agg({'Amount': 'sum', 'CustomerID': 'nunique'}), rsuffix='previous').reset_index(names='Day_list').fillna(0)
t2

,Day_list,Amount,CustomerID,Amountprevious,CustomerIDprevious
0,1,0.00,0.0,0.00,0.0
1,2,0.00,0.0,0.00,0.0
2,3,0.00,0.0,1823.34,3.0
3,4,0.00,0.0,0.00,0.0
4,5,3442.51,3.0,236.00,1.0
5,6,371.45,1.0,0.00,0.0
6,7,0.00,0.0,0.00,0.0
7,8,1311.74,3.0,1056.43,1.0
8,9,1030.59,2.0,537.90,3.0
9,10,480.00,1.0,489.10,1.0


In [13]:
t3 = current_month.groupby('CustomerID')[['Quantity', 'Amount']].sum().reset_index(names='CustomerID').sort_values('Amount', ascending=False)[:5]
t3

,CustomerID,Quantity,Amount
5,12567.0,773,1936.44
12,12678.0,1171,1899.32
23,12731.0,514,1359.58
22,12727.0,363,793.08
14,12682.0,313,785.09


In [14]:
t4 = current_month.groupby('Description')[['Quantity', 'Amount']].sum().reset_index(names='Items').sort_values('Amount', ascending=False)[:5]
t4

,Items,Quantity,Amount
199,POSTAGE,77,1386.0
200,RABBIT NIGHT LIGHT,258,474.0
118,JUMBO BAG DOILEY PATTERNS,230,420.4
230,REVOLVER WOODEN RULER,192,374.4
169,PAPER BUNTING RETROSPOT,116,310.2


In [15]:
t5 = current_month.groupby(['Description', 'CustomerID'])['Amount'].sum().reset_index()
t5

,Description,CustomerID,Amount
0,SET 2 TEA TOWELS I LOVE LONDON,12681.0,19.50
1,SPACEBOY BABY GIFT SET,12567.0,67.80
2,SPACEBOY BABY GIFT SET,12721.0,16.95
3,SPACEBOY BABY GIFT SET,12731.0,89.70
4,12 MESSAGE CARDS WITH ENVELOPES,12615.0,19.80
...,...,...,...
545,WRAP DOILEY DESIGN,12721.0,10.50
546,WRAP ENGLISH ROSE,12721.0,10.50
547,WRAP GREEN PEARS,12567.0,10.50
548,WRAP PINK FAIRY CAKES,12721.0,10.50


In [16]:
t6 = t5.groupby('Description')['Amount'].sum().reset_index()
t6

,Description,Amount
0,SET 2 TEA TOWELS I LOVE LONDON,19.50
1,SPACEBOY BABY GIFT SET,174.45
2,12 MESSAGE CARDS WITH ENVELOPES,19.80
3,12 PENCILS SMALL TUBE SKULL,31.20
4,12 PENCILS TALL TUBE RED RETROSPOT,10.20
...,...,...
315,WRAP DOILEY DESIGN,10.50
316,WRAP ENGLISH ROSE,10.50
317,WRAP GREEN PEARS,10.50
318,WRAP PINK FAIRY CAKES,10.50


In [17]:
t7 =  t5.groupby('CustomerID')['Amount'].sum().reset_index()
t7

,CustomerID,Amount
0,12437.0,501.64
1,12490.0,484.06
2,12493.0,201.24
3,12535.0,371.45
4,12562.0,510.27
5,12567.0,1936.44
6,12583.0,617.48
7,12599.0,134.28
8,12615.0,634.74
9,12624.0,299.94


Adding all tables into one list


In [20]:
tables_set = [t1, t2, t3, t4, t5, t6, t7]

Initializing an instance of the class. We should put a path to the template and path for the target report.  


In [21]:
rc = ReportCreator(template_path='template.xlsx', report_path='sales_report.xlsx', log_path='no log')

The method 'write' does all operations to create the target report automatically.

In [22]:
rc.write(variables=params, tables=tables_set)

Processing template .\template.xlsx...
2 sheets found
Processing sheet: review
Analysing for pivot table...
No pivot tables
Writing a report...
Finished
Processing sheet: details
Analysing for pivot table...
Successfully found 1 pivot_tables
Writing a report...
Finished
--------------------------------------------------------------------------------------------------------------------------
Status: Success
--------------------------------------------------------------------------------------------------------------------------
Template_path: .\template.xlsx
Report_path: .\sales_report.xlsx
Log_path: None
See `self.writing_history` for details


We also can see errors and writing history of this operation if something goes wrong.

Check the results :)